In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import tokenize
import re
from unidecode import unidecode
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from itertools import chain

In [38]:
STOP_WORDS = chain(nltk.corpus.stopwords.words("english"), 
                  nltk.corpus.stopwords.words("portuguese"))
MAX_FEATURES = [50, None, 500, 10000]
SOLVERS = ['lbfgs','liblinear','newton-cg','sag','saga']

In [39]:
def add_stop_words(*args):
    for arg in args:
        STOP_WORDS.append(arg)
    return STOP_WORDS
def normalize_sentence(sentence):
    norm_sentence = sentence.lower()
    norm_sentence = re.sub(r'[^\w\s]','', norm_sentence) 
    norm_sentence = norm_sentence.strip() 
    norm_sentence = unidecode(norm_sentence)
    norm_sentence = ' '.join(norm_sentence.split()) 
    return norm_sentence

In [44]:
xinput = ['terceira guerra mundial vem se aproximando', 'data science é muito útil para lidar com dados', 'agora o elon musk comprou o twitter', 'manhã com chuva', 'as acoes da bolsa subiram muito', 'bitcoin baixou', 'voce vai viajar?']
yinput = [-1, 0, 1, 0, 1, -1, 0]

In [45]:
white_space_tokenize = tokenize.WhitespaceTokenizer()  
phrase_pos = list()
for text in tqdm(xinput):
    new_phrase = list()
    phrase_text = white_space_tokenize.tokenize(
        text #normalize_sentence(text)
    )
    for word in phrase_text:
        if word not in STOP_WORDS:
            stemmer = nltk.RSLPStemmer()
            norm_word = stemmer.stem(word)
            new_phrase.append(norm_word)

    phrase_pos.append(' '.join(new_phrase))
xinput_pos = phrase_pos

100%|████████████████████████████████████████████| 7/7 [00:00<00:00, 165.73it/s]


In [32]:
vectorizer = CountVectorizer(max_features=500)
bag_of_words = vectorizer.fit_transform(xinput_pos)
train, test, class_train, class_test =\
        train_test_split(bag_of_words, yinput, test_size=0.2, random_state=4)
logistic_regression = LogisticRegression(solver='newton-cg')
logistic_regression.fit(train, class_train)
logistic_regression.predict(bag_of_words)
#logistic_regression.score(bag_of_words, yinput)

array([-1,  1, -1,  0])

In [33]:
logistic_regression.score(bag_of_words,yinput)

0.75

In [46]:
def test_solvers(solver):
    vectorizer = CountVectorizer()
    bag_of_words = vectorizer.fit_transform(xinput_pos)
    train, test, class_train, class_test =\
            train_test_split(bag_of_words, yinput, test_size=0.2, random_state=4)
    logistic_regression = LogisticRegression(solver=solver)
    logistic_regression.fit(train, class_train)
    print(logistic_regression.predict(bag_of_words))
    print(logistic_regression.score(bag_of_words,yinput))
for solver in SOLVERS:
    test_solvers(solver)

[-1  0  1  0 -1 -1 -1]
0.7142857142857143
[-1  0  1  0  0 -1 -1]
0.7142857142857143
[-1  0  1  0 -1 -1 -1]
0.7142857142857143
[-1  0  1  0  0 -1 -1]
0.7142857142857143
[-1  0  1  0  0 -1 -1]
0.7142857142857143


/home/marten/Desktop/workdir/ALURA/NLP/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [47]:
yinput

[-1, 0, 1, 0, 1, -1, 0]